# 决策树
决策树(decision tree)是一种基本的分类和回归方法.在分类问题中,决策树在对特征进行分类过程中可以被认为是一种 `if-then` 规则的集合.  
决策树优点:  
* 有良好的可读性
* 分类速度快  

决策树的学习包括:  
* 特征选择
* 决策树的生成
* 决策树的修剪  

决策树的算法:  
* ID3 : 信息增益
* C4.5 : 信息增益比
* CART : gini 指数

## 特征选择
### 1. 信息增益选择特征
选择合适的特征作为根节点能提升分类效果,特征的信息增益最大的为最优特征.  
信息增益 = 经验熵 - 条件经验熵  
$ g(D,A) = H(D) - H(D|A) $  
* 经验熵: H(D) 为数据集 D 的经验熵  
$ H(D) = -\sum_{i=1}^{n}p_i{log_2{p_i}} $  
$ p_i = P(X=x_i),i=1,2,...,n $  
* 经验条件熵: H(Y|X) 特征 X 对数据集 D 的经验熵  
$ H(Y|X) = \sum_{i=1}^{n}p_iH(Y|X=x_i) $  

示例:  
训练数据集 D,|D| 表示其样本容量,即样本个数.设 K 个类 $ C_k $,k=1,2,...,K,$ |C_k| $ 为属于类 $ C_k $ 的样本个数,$ \sum_{k=1}^{K}|C_k|=|D| $.社特征 A 有 n 个不同的取值 {$ a_1,a_2,...,a_n $},根据特征 A 的取值将 D 划分为 n 个子集 {$ D_1,D_2,...,D_3 $},$ |D_i| $ 为 $ D_i $ 的样本个数,$ \sum_{i=1}^{n}|D_i|=|D| $,子集 $ D_i $ 中数据类 $ C_k $ 的样本的集合为 $ D_{ik} $,即 $ D_{ik}={D_i}\bigcap{C_k},{D_{ik}} 为 D_{ik} $ 的样本个数.  
1. 计算数据集 D 的经验熵 H(D)  
$ H(D) = -\sum_{k=1}^{k}\frac{|C_k|}{|D|}log_2\frac{|C_k|}{|D|} $  
2. 计算特征 A 对数据集 D 的经验条件熵 H(D|A)  
$ H(D|A) = \sum_{i=1}^{n}\frac{|C_i|}{|D|}H(D_i) = -\sum_{i=1}^{n}\frac{|C_i|}{|D|}\sum_{k=1}^{k}\frac{|D_ik|}{|Di|}log_2\frac{|D_ik|}{|Di|} $  
3. 计算信息增益  
$ g(D,A) = H(D) - H(D|A) $  

### 2. 信息增益比选择特征  
信息增益比是对信息增益的校正.当分类困难,训练数据集的经验熵过大,信息增益值会偏大,反之偏小,这时就需要对其进行校正.  
* 信息增益比 = 信息增益 / 经验熵  
$ g_R(D,A) = \frac{g(D,A)}{H(D)} $  

## 决策树的生成  
